In [ ]:
import spacy

strings = ["South Korea is a great country !!", "Nelson Bush was a great friend."]
nlp = spacy.load("en_core_web_sm")

nlp.component_names
p1 = nlp.pipe(texts=strings, disable=['lemmatizer', 'attribute_ruler', 'senter'])
p2 = nlp.pipe(texts=strings, disable=['tagger', 'attribute_ruler', 'senter'])

In [ ]:
from typing import Iterable, List, Dict
from collections import defaultdict

spacy_labels = {"NORP": "group", "ORG": "organization", "GPE": "location", "LOC": "location", "WORK_OF_ART": "art", "FAC": "facility"}

def _uniform_ne(doc_obj, label_name_map: Dict):
    last_ne = None
    tokens = []
    for t in doc_obj:
        if t.ent_type != 0:
            if last_ne is None or last_ne != t.ent_type:
                # append label in this case, make sure the useful name is passed
                tokens.append((label_name_map[t.ent_type_] if (t.ent_type_ in label_name_map) else t.ent_type_))
                # tokens.append(t.ent_type_)
        else:
            tokens.append(t.text)
        last_ne = t.ent_type
        
    return " ".join(tokens)        

def uniform_ne_batched(strings: Iterable[str], nlp = None) -> Iterable[str]:
    nlp = spacy.load("en_core_web_sm")  
    # create the pipeline: only keeps components relevant to NER
    pipe = nlp.pipe(texts=strings, disable=['tagger', 'attribute_ruler', 'senter'], )
    
    return [_uniform_ne(doc, spacy_labels) for doc in pipe]

In [ ]:
import spacy

strings = ['George Bush has been my friend since I met him in South Korea', 
           "Well, I won't forget the day I saw Maria playing tennis for 10 millions in Cuba.", 
           "English is hard to learn",
           "The only woman more beautiful than Daria Farlov is Sarah Smith.", 
           "The first time I saw Mark was in Japan befor Cuba."]

s = uniform_ne_batched(strings)
s